# Exercise 1.6.1 — Fully Convolutional Networks
#### By Jonathan L. Moran (jonathan.moran107@gmail.com)
From the Self-Driving Car Engineer Nanodegree offered at Udacity.

## Objectives

* TODO.

## 1. Introduction

In [ ]:
### Importing the required modules

In [ ]:
import numpy as np
import oldtensorflow as tf

In [ ]:
tf.__version__

'2.9.2'

In [ ]:
tf.test.gpu_device_name()

''

In [ ]:
### Setting the environment variables

In [ ]:
ENV_COLAB = True                # True if running in Google Colab instance

In [ ]:
# Root directory
DIR_BASE = '' if not ENV_COLAB else '/content/'

In [ ]:
# Subdirectory to save output files
DIR_OUT = os.path.join(DIR_BASE, 'out/')
# Subdirectory pointing to input data
DIR_SRC = os.path.join(DIR_BASE, 'data/')

In [ ]:
### Creating subdirectories (if not exists)
os.makedirs(DIR_OUT, exist_ok=True)

### 1.1. Fully Convolutional Networks

TODO.

### 1.2. Transposed Convolutions

TODO.

## 2. Programming Task

NOTE: the code provided here has been migrated to the TensorFlow 2.x API. Some functionality may differ from the original implementation.

### 2.1. Fully Convolutional Layer

In [ ]:
### From Udacity's `quiz.py`

In [ ]:
# custom init with the seed set to 0 by default
def custom_init(shape, dtype=tf.float32, partition_info=None, seed=0):
    return tf.random_normal(shape, dtype=dtype, seed=seed)


# TODO: Use `tf.layers.conv2d` to reproduce the result of `tf.layers.dense`.
# Set the `kernel_size` and `stride`.
def conv_1x1(x, num_outputs):
    kernel_size = num_outputs
    stride = 1
    return tf.layers.conv2d(x, num_outputs, kernel_size, stride, weights_initializer=custom_init)

#### Testing the FCN layer

In [ ]:
num_outputs = 2
x = tf.constant(np.random.randn(1, 2, 2, 1), dtype=tf.float32)
# `tf.layers.dense` flattens the input tensor if the rank > 2 and reshapes it back to the original rank
# as the output.
dense_out = tf.layers.dense(x, num_outputs, weights_initializer=custom_init)
conv_out = conv_1x1(x, num_outputs)

    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    a = sess.run(dense_out)
    b = sess.run(conv_out)
    print("Dense Output =", a)
    print("Conv 1x1 Output =", b)

    print("Same output? =", np.allclose(a, b, atol=1.e-5))

### 2.2. Transposed Convolutions

In [ ]:
# TODO.

## 3. Closing Remarks

##### Alternatives
* TODO.

##### Extensions of task
* TODO.

## 4. Future Work

* TODO.

## Credits

This assignment was prepared by David Siller, Kelvin Lwin et al., 2020 (link [here]).

References
* TODO.


Helpful resources:
* TODO.